In [1]:
import numpy as np

In [2]:
def precision(relevance_query: list) -> float:
    relevance_vector = np.array(relevance_query)
    return np.sum(relevance_vector) / len(relevance_vector)

def precision_at_k(relevance_query: list, k: int) -> float:
    relevance_vector = np.array(relevance_query)
    relevance_vector_at_k = relevance_vector[:k]
    return np.sum(relevance_vector_at_k) / k if k > 0 else 0.0

def recall_at_k(relevance_query: list,
                number_relevant_docs: int,
                k: int) -> float:
    relevance_vector = np.array(relevance_query)
    relevance_vector_at_k = relevance_vector[:k]
    return np.sum(relevance_vector_at_k) / number_relevant_docs if number_relevant_docs > 0 else 0.0

def average_precision(relevance_query: list) -> float:
    """
    Calcula la Average Precision (AP) para una consulta dada usando NumPy.

    Args:
        relevance_query: Una lista de enteros (0 o 1) que indica la relevancia de cada documento.
                         1 significa relevante, 0 significa no relevante.

    Returns:
        El valor de Average Precision.
    """
    relevance_vector = np.array(relevance_query)
    relevant_indices = np.where(relevance_vector == 1)[0]
    
    if relevant_indices.size == 0:
        return 0.0

    precision_sum = 0.0
    for index in relevant_indices:
        precision_sum += precision_at_k(relevance_query, index + 1)

    return precision_sum / len(relevant_indices)

def mean_average_precision(vector_list):
    sum_average_precision = 0.0
    for vector in vector_list:
        sum_average_precision += average_precision(vector)

    return sum_average_precision/len(vector_list)

def dcg_at_k(relevance_query: list, k: int) -> float:
    relevance_vector = np.array(relevance_query)
    relevance_vector_at_k = relevance_vector[:k]
    dcg_at_k = 0.0
    for index, relevance in enumerate(relevance_vector_at_k):
        dcg_at_k += relevance/np.log2(max(index+1,2))
    return dcg_at_k

def ndcg_at_k(relevance_query: list, k: int) ->float:
    relevance_vector = np.array(relevance_query)
    relevance_vector_at_k = relevance_vector[:k]
    dcg_at_k = 0.0
    
    for index, relevance in enumerate(relevance_vector_at_k):
        dcg_at_k += relevance/np.log2(max(index+1,2))
    
    ordered_relevance_vector = np.sort(relevance_vector)[::-1]
    ordered_relevance_vector_at_k= ordered_relevance_vector[:k]
    best_dcg_at_k = 0.0
    
    for index, relevance in enumerate(ordered_relevance_vector_at_k):
        best_dcg_at_k += relevance/np.log2(max(index+1,2))

    return dcg_at_k/best_dcg_at_k

